# Social Network Recommendation System - Demo

This notebook demonstrates the key features of our LinkedIn-style professional network built with Neo4j and FastAPI.

In [1]:
# Import required libraries
import requests
from neo4j import GraphDatabase
from pprint import pprint

API_BASE_URL = "http://localhost:8000"
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password123"

print("✓ Imports successful")

✓ Imports successful


## 1. Health Check

In [2]:
response = requests.get(f"{API_BASE_URL}/health")
print("API Health Check:")
pprint(response.json())

API Health Check:
{'database': 'connected', 'health_check': True, 'status': 'healthy'}


## 2. Database Statistics

In [3]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def get_database_stats(tx):
    query = """
    MATCH (u:User) WITH COUNT(u) AS users
    MATCH (s:Skill) WITH users, COUNT(s) AS skills
    MATCH (c:Company) WITH users, skills, COUNT(c) AS companies
    MATCH ()-[knows:KNOWS]-() WITH users, skills, companies, COUNT(knows)/2 AS connections
    MATCH ()-[has:HAS_SKILL]->() WITH users, skills, companies, connections, COUNT(has) AS user_skills
    MATCH ()-[works:WORKS_AT]->() 
    RETURN users, skills, companies, connections, user_skills, works AS employments
    """
    result = tx.run(query)
    return result.single()

with driver.session() as session:
    stats = session.execute_read(get_database_stats)
    print("\n📊 Database Statistics:")
    print(f"  Users:       {stats['users']}")
    print(f"  Skills:      {stats['skills']}")
    print(f"  Companies:   {stats['companies']}")
    print(f"  Connections: {int(stats['connections'])}")
    print(f"  User-Skills: {stats['user_skills']}")
    print(f"  Employments: {stats['employments']}")


📊 Database Statistics:
  Users:       101
  Skills:      30
  Companies:   8
  Connections: 2739
  User-Skills: 633
  Employments: <Relationship element_id='5:eeddc835-7742-4475-b188-2a6eac5be74e:630' nodes=(<Node element_id='4:eeddc835-7742-4475-b188-2a6eac5be74e:50' labels=frozenset() properties={}>, <Node element_id='4:eeddc835-7742-4475-b188-2a6eac5be74e:1' labels=frozenset() properties={}>) type='WORKS_AT' properties={}>


c:\Users\walid\OneDrive\Desktop\Graph-knowledge-Final\.venv\Lib\site-packages\neo4j\_sync\work\result.py:497: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "


## 3. Find a Well-Connected User

Let's find a user with lots of connections to demonstrate recommendations:

In [7]:
# Find user with most connections
def get_well_connected_user(tx):
    query = """
    MATCH (u:User)-[:KNOWS]-(conn:User)
    WITH u, COUNT(conn) AS connection_count
    WHERE connection_count > 10 AND connection_count < 50
    RETURN u.user_id AS user_id, u.name AS name, u.title AS title, u.email AS email,
           connection_count
    ORDER BY connection_count DESC
    LIMIT 1
    """
    result = tx.run(query)
    return result.single()

with driver.session() as session:
    user = session.execute_read(get_well_connected_user)
    if user:
        sample_user_id = user['user_id']
        sample_user_name = user['name']
        print(f"\n👤 Selected User: {user['name']}")
        print(f"   Title: {user['title']}")
        print(f"   Email: {user['email']}")
        print(f"   Connections: {user['connection_count']}")
        print(f"   User ID: {sample_user_id}")
    else:
        print("⚠️ No well-connected users found. Make sure data is seeded!")
        sample_user_id = None
        sample_user_name = "Unknown"


👤 Selected User: Anne Diaz
   Title: Full Stack Developer
   Email: maylori@example.org
   Connections: 33
   User ID: 8eed70d9-e8f3-432d-9e01-e0e11fbb6afa


## 4. Friend Recommendations

Find potential connections based on mutual friends and common skills:

In [8]:
if sample_user_id:
    response = requests.get(
        f"{API_BASE_URL}/api/users/{sample_user_id}/recommendations/friends?limit=5"
    )
    recommendations = response.json()

    print(f"\n🤝 Friend Recommendations for {sample_user_name}:")
    
    if recommendations:
        for i, rec in enumerate(recommendations, 1):
            print(f"\n{i}. {rec['name']} - {rec.get('title', 'N/A')}")
            print(f"   Score: {rec['score']:.2f}")
            print(f"   Mutual Connections: {rec['mutual_connections']}")
            if rec['common_skills']:
                print(f"   Common Skills: {', '.join(rec['common_skills'][:3])}")
            print(f"   Reason: {rec['reason']}")
    else:
        print("  No recommendations found (user might already know everyone!)")
else:
    print("⚠️ Skipping - no user selected")


🤝 Friend Recommendations for Anne Diaz:

1. Howard Brown - Security Engineer
   Score: 1.00
   Mutual Connections: 33
   Reason: Many mutual connections

2. Nicole Williams - Database Administrator
   Score: 1.00
   Mutual Connections: 33
   Reason: Many mutual connections

3. Jennifer Kelley - Product Manager
   Score: 1.00
   Mutual Connections: 33
   Reason: Many mutual connections

4. Emily Preston - Senior Software Engineer
   Score: 1.00
   Mutual Connections: 33
   Reason: Many mutual connections

5. Kimberly Chambers - UX Designer
   Score: 1.00
   Mutual Connections: 33
   Reason: Many mutual connections


## 5. Job Recommendations

Find job opportunities based on skill matching:

In [9]:
if sample_user_id:
    response = requests.get(
        f"{API_BASE_URL}/api/users/{sample_user_id}/recommendations/jobs?limit=5"
    )
    jobs = response.json()

    print(f"\n💼 Job Recommendations for {sample_user_name}:")
    
    if jobs:
        for i, job in enumerate(jobs, 1):
            print(f"\n{i}. {job['title']}")
            print(f"   Company: {job['company']}")
            print(f"   Location: {job.get('location', 'N/A')}")
            print(f"   Skill Match: {job['skill_match_rate']*100:.0f}%")
            if job['matching_skills']:
                print(f"   Matching Skills: {', '.join(job['matching_skills'][:5])}")
    else:
        print("  No job recommendations found (user might not have skills or work at all companies)")
else:
    print("⚠️ Skipping - no user selected")


💼 Job Recommendations for Anne Diaz:
  No job recommendations found (user might not have skills or work at all companies)


## 6. People You May Know

Discover professionals with shared connections, skills, or company:

In [10]:
if sample_user_id:
    response = requests.get(
        f"{API_BASE_URL}/api/users/{sample_user_id}/suggestions/people?limit=5"
    )
    suggestions = response.json()

    print(f"\n👋 People You May Know:")
    
    if suggestions:
        for i, person in enumerate(suggestions, 1):
            print(f"\n{i}. {person['name']} - {person.get('title', 'N/A')}")
            print(f"   Company: {person.get('company', 'N/A')}")
            print(f"   Score: {person['score']:.2f}")
            print(f"   Mutual Connections: {person['mutual_connections']}")
            print(f"   Common Skills: {person['common_skills']}")
            if person['same_company']:
                print(f"   ⭐ Works at the same company!")
    else:
        print("  No suggestions found")
else:
    print("⚠️ Skipping - no user selected")


👋 People You May Know:

1. Shaun Huff - Cloud Architect
   Company: None
   Score: 1.00
   Mutual Connections: 33
   Common Skills: 0

2. Marissa Moore - Senior Data Scientist
   Company: None
   Score: 1.00
   Mutual Connections: 33
   Common Skills: 0

3. Tanya Fletcher MD - Full Stack Developer
   Company: None
   Score: 1.00
   Mutual Connections: 33
   Common Skills: 0

4. David Taylor - Senior Data Scientist
   Company: None
   Score: 1.00
   Mutual Connections: 33
   Common Skills: 0

5. Thomas Solis - QA Engineer
   Company: None
   Score: 1.00
   Mutual Connections: 33
   Common Skills: 0


## 7. Shortest Path Between Users

Find how two professionals are connected:

In [11]:
# Get two different users
def get_two_users(tx):
    query = """
    MATCH (u:User)
    RETURN u.user_id AS user_id, u.name AS name
    LIMIT 2
    """
    result = tx.run(query)
    return [record.data() for record in result]

with driver.session() as session:
    users = session.execute_read(get_two_users)
    
if len(users) >= 2:
    user1_id = users[0]['user_id']
    user2_id = users[1]['user_id']

    response = requests.get(
        f"{API_BASE_URL}/api/paths/shortest?from={user1_id}&to={user2_id}"
    )
    path = response.json()

    print(f"\n🔗 Shortest Path:")
    print(f"   From: {users[0]['name']}")
    print(f"   To: {users[1]['name']}")
    print(f"   Path Length: {path['path_length']} hop(s)")
    print(f"   Path Exists: {path['exists']}")

    if path['exists'] and path['nodes']:
        print(f"\n   Connection Path:")
        for i, node in enumerate(path['nodes']):
            print(f"   {i+1}. {node['name']} ({node.get('title', 'N/A')})")


🔗 Shortest Path:
   From: Kayla Rasmussen
   To: Thomas Solis
   Path Length: 1 hop(s)
   Path Exists: True

   Connection Path:
   1. Kayla Rasmussen (QA Engineer)
   2. Thomas Solis (QA Engineer)


## 8. Most Connected Users

In [12]:
def get_most_connected_users(tx):
    query = """
    MATCH (u:User)-[:KNOWS]-(connection)
    WITH u, COUNT(connection) AS connections
    RETURN u.name AS name, u.title AS title, connections
    ORDER BY connections DESC
    LIMIT 10
    """
    result = tx.run(query)
    return [record.data() for record in result]

with driver.session() as session:
    top_users = session.execute_read(get_most_connected_users)
    print("\n⭐ Most Connected Users:")
    for i, user in enumerate(top_users, 1):
        print(f"{i}. {user['name']} ({user['title']}) - {user['connections']} connections")


⭐ Most Connected Users:
1. William Hancock (Backend Developer) - 99 connections
2. Gina Ballard (Engineering Manager) - 99 connections
3. Deborah Carter (Staff Software Engineer) - 99 connections
4. Andres Fowler (Database Administrator) - 99 connections
5. Janet Moss (Product Manager) - 99 connections
6. Tara Wade (Senior Software Engineer) - 99 connections
7. Stephanie Hall (QA Engineer) - 99 connections
8. Kayla Rasmussen (QA Engineer) - 99 connections
9. Keith Cordova (Backend Developer) - 99 connections
10. Amanda Moore (Backend Developer) - 99 connections


## 9. Popular Skills

In [13]:
def get_popular_skills(tx):
    query = """
    MATCH (u:User)-[:HAS_SKILL]->(s:Skill)
    WITH s, COUNT(u) AS users
    RETURN s.name AS skill, s.category AS category, users
    ORDER BY users DESC
    LIMIT 10
    """
    result = tx.run(query)
    return [record.data() for record in result]

with driver.session() as session:
    popular_skills = session.execute_read(get_popular_skills)
    print("\n🔥 Most Popular Skills:")
    for i, skill in enumerate(popular_skills, 1):
        print(f"{i}. {skill['skill']} ({skill['category']}) - {skill['users']} users")


🔥 Most Popular Skills:
1. Python (Programming) - 22 users
2. FastAPI (Backend) - 22 users
3. Neo4j (Database) - 22 users
4. Java (Programming) - 21 users
5. Rust (Programming) - 21 users
6. React (Frontend) - 21 users
7. Go (Programming) - 21 users
8. Angular (Frontend) - 21 users
9. Node.js (Backend) - 21 users
10. Vue.js (Frontend) - 21 users


## 10. Company Network Density

In [14]:
def get_company_network_density(tx):
    query = """
    MATCH (c:Company)<-[:WORKS_AT]-(u:User)
    WITH c, COUNT(u) AS employees
    MATCH (c)<-[:WORKS_AT]-(e1:User)-[:KNOWS]-(e2:User)-[:WORKS_AT]->(c)
    WHERE e1 <> e2
    WITH c, employees, COUNT(DISTINCT e1) AS connected_employees
    WHERE connected_employees > 0
    RETURN c.name AS company, employees, connected_employees,
           toFloat(connected_employees) / employees AS density
    ORDER BY density DESC
    """
    result = tx.run(query)
    return [record.data() for record in result]

with driver.session() as session:
    companies = session.execute_read(get_company_network_density)
    print("\n🏢 Company Network Density:")
    
    if companies:
        for i, company in enumerate(companies, 1):
            print(f"\n{i}. {company['company']}")
            print(f"   Employees: {company['employees']}")
            print(f"   Internal Connections: {company['connected_employees']}")
            print(f"   Network Density: {company['density']*100:.1f}%")
    else:
        print("  No companies with internal connections found")


🏢 Company Network Density:
  No companies with internal connections found


## Cleanup

In [15]:
driver.close()
print("✓ Database connection closed")
print("\n🎉 Demo complete!")

✓ Database connection closed

🎉 Demo complete!


## Summary

This demo showcased:
1. ✅ Health check and system status
2. ✅ Database statistics
3. ✅ Finding well-connected users
4. ✅ Friend recommendations (graph traversal)
5. ✅ Job recommendations (skill matching)
6. ✅ People suggestions (multi-signal scoring)
7. ✅ Shortest path finding
8. ✅ Network analytics
9. ✅ Skill popularity analysis
10. ✅ Company network density metrics